In [43]:
import pandas as pd
%pylab inline

def u(x,t):
    return np.exp(x+t)

def f(x,t):
    return 1/2*np.exp(x+t)

def alpha(t):
    return np.exp(t)

def beta(t):
    return np.exp(t+1) # du/dx = exp(t)

def phi(x):
    return np.exp(x)


Populating the interactive namespace from numpy and matplotlib


C:\Users\79266\Anaconda3\lib\site-packages\IPython\core\magics\pylab.py:159: UserWarning:

pylab import has clobbered these variables: ['beta', 'f']
`%matplotlib` prevents importing * from pylab and numpy



In [53]:
x_l = 0.0
x_r = 1.0
t_l = 0.0
t_r = 3

h = 0.01
tau = 0.01

N = int((x_r - x_l) / h)+1
K = int((t_r - t_l) / tau)+1


x = [i * h for i in range(N)] 
t = [j*tau for j in range(K)]


matrix_k = np.zeros((len(t),len(x)))

for i in range(len(t)):
    for j in range(len(x)):
        matrix_k[i][j] = u(x[j],t[i])

In [54]:
# домнож на 2, чтобы увел общее кол-во интервалов на слое (см ниже)
N = int((x_r - x_l) / h)+1
K = int((t_r - t_l) / tau)+1


x = [i * h for i in range(N)] 
t = [j*tau for j in range(K)]


y_prev = [phi(t[i]) for i in range(N)] # u(x,0) - заполненние "целых" слоев
y_prev_1 = [alpha(t[i]) for i in range(K)] # u(0,t) - заполненние "целых" слоев

matrix_ksi = np.zeros((len(t),len(x))) # матрица кси(j),len(x)-2 --> от 1 до N-1 точки   

matrix_1 = np.zeros((K,N)) # K - строк - время, N - столбцов - координата

matrix_1[:,0] = y_prev_1 # u(0,t)
matrix_1[0,:] = y_prev # u(x,0)
matrix_1[:,-1] = [beta(t[i]) for i in range(len(np.array(t)))] #заполнение u(1,t) в целых полуслоях

a = c = - 0.5 * tau /(2* h**2) # k=sigma=1
b = 1 +  0.5 * 0.01/(h**2)

################################## alpha, beta
X = np.zeros(len(x)-1)

X[0] = 0
for i in range(1,X.shape[0]):
    X[i] = -a/(b+c*X[i-1])
################################## alpha, beta

matrix_ans = np.zeros((len(t),len(x)))
matrix_ans[0] = matrix_1[0] 
for i in range(1,matrix_ans.shape[0]):
    
    g = np.zeros(len(x)-1)
    matrix_ksi = np.zeros(len(x))

    for j in range(1,matrix_ksi.shape[0]-1):
        matrix_ksi[j] = matrix_ans[i-1][j] + abs(a)*(matrix_ans[i-1][j+1] - 2*matrix_ans[i-1][j] + matrix_ans[i-1][j-1]) + 0.01*f(x[j],t[i-1])
    
    matrix_ksi = np.delete(matrix_ksi, 0)
    matrix_ksi = np.delete(matrix_ksi, -1)
                          
    g[0] = matrix_1[i][0] # гранич левое условие
    
    for j in range(1,g.shape[0]):
        g[j] = (matrix_ksi[j-1]-c*g[j-1])/(b+c*X[j-1])
        
    matrix_ans[i][-1] = matrix_1[i][-1]
                          
    for j in reversed(range(1,len(x))): #[1,100]
        matrix_ans[i][j-1] = X[j-1]*matrix_ans[i][j] + g[j-1]


In [55]:
import plotly.express as px
import plotly.graph_objs as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=x,y=matrix_k[-5],mode='lines+markers',name='Аналитическое решение'))
fig.add_trace(go.Scatter(x=x,y=matrix_ans[-5],mode='lines+markers',name='Численное решение'))
fig.update_layout(title='Метод Кранка-Николсона')
fig.show()

In [57]:
matrix_ans[0][:]

array([1.        , 1.01005017, 1.02020134, 1.03045453, 1.04081077,
       1.0512711 , 1.06183655, 1.07250818, 1.08328707, 1.09417428,
       1.10517092, 1.11627807, 1.12749685, 1.13882838, 1.1502738 ,
       1.16183424, 1.17351087, 1.18530485, 1.19721736, 1.2092496 ,
       1.22140276, 1.23367806, 1.24607673, 1.25860001, 1.27124915,
       1.28402542, 1.29693009, 1.30996445, 1.32312981, 1.33642749,
       1.34985881, 1.36342511, 1.37712776, 1.39096813, 1.40494759,
       1.41906755, 1.43332941, 1.44773461, 1.46228459, 1.47698079,
       1.4918247 , 1.50681779, 1.52196156, 1.53725752, 1.55270722,
       1.56831219, 1.58407398, 1.59999419, 1.6160744 , 1.63231622,
       1.64872127, 1.66529119, 1.68202765, 1.69893231, 1.71600686,
       1.73325302, 1.7506725 , 1.76826705, 1.78603843, 1.80398842,
       1.8221188 , 1.8404314 , 1.85892804, 1.87761058, 1.89648088,
       1.91554083, 1.93479233, 1.95423732, 1.97387773, 1.99371553,
       2.01375271, 2.03399126, 2.05443321, 2.07508061, 2.09593

#### Доп задача 

$$\frac{\partial u}{\partial t} = a\frac{\partial^{2} u}{\partial x ^ {2}} + f$$

$$u(x,0) = \varphi(x),  0 \leq x \leq 1 $$

$$u(1,t) = \beta(t)$$
$$u(0,t) = \alpha(t)$$

$$\frac{\partial u}{\partial x} \mid_{x=0}  = \nu(t)$$

##### Изначально:
$$a_j=c_j=-\frac{k\Delta t}{2h^2},    b_j = 1+\frac{k\Delta t}{h^2}$$
$$\xi ^{n}_j = u^n_j+\frac{k\Delta t}{2h^2}(u^n_{j+1}-2u^n_j+u^n_{j-1})+f(t^n,x_j)\Delta t$$

$$X_{j+1} = \frac{-a}{b+cX_j},  Z_{j+1}=\frac{\xi_j-cZ_j}{b+cX_j}$$

$$y_{j-1}=X_jy_j+Z_j$$

$$\frac{\partial u}{\partial x}\mid_{x=0} = \mu(t)$$

In [207]:
def nu(t):
    return 0#np.exp(t) #- 0.09


h = 0.01
tau = 0.01


N = int((x_r - x_l) / h)+1
K = int((t_r - t_l) / tau)+1


x = [i * h for i in range(N)] 
t = [j*tau for j in range(K)]


matrix_k = np.zeros((len(t),len(x)))

for i in range(len(t)):
    for j in range(len(x)):
        matrix_k[i][j] = u(x[j],t[i])




# домнож на 2, чтобы увел общее кол-во интервалов на слое (см ниже)
N = int((x_r - x_l) / h)+1
K = int((t_r - t_l) / tau)+1


x = [i * h for i in range(N)] 
t = [j*tau for j in range(K)]

matrix_ksi = np.zeros((len(t),len(x))) # матрица кси(j),len(x)-2 --> от 1 до N-1 точки   

matrix_1 = np.zeros((K,N)) # K - строк - время, N - столбцов - координата

matrix_1[:,0] = [alpha(t[i]) for i in range(K)] # u(0,t) - заполненние "целых" слоев
matrix_1[0,:] = [0. for i in range(N)] # u(x,0) - заполненние "целых" слоев
matrix_1[:,-1] = [beta(t[i]) for i in range(K)] #заполнение u(1,t) в целых полуслоях



a = c = - 0.5 * tau /(2* h**2) # k=sigma=0.5
b = 1 +  0.5 * tau/(h**2)


X = np.zeros(len(x)-1)

#X[0] = 0
X[0]=tau*0.5/(h**2 + 0.5*tau)
for j in range(1,X.shape[0]): #прямой ход прогонки
    X[j] = -a/(b+c*X[j-1])                         ### X


matrix_ans = np.zeros((len(t),len(x)))
matrix_ans[0] = matrix_1[0] 
for i in range(1,matrix_ans.shape[0]):
    
    g = np.zeros(len(x)-1)
    matrix_ksi = np.zeros(len(x))

    for j in range(1,matrix_ksi.shape[0]-1):
        matrix_ksi[j] = matrix_ans[i-1][j] + abs(a)*(matrix_ans[i-1][j+1] - 2*matrix_ans[i-1][j] + matrix_ans[i-1][j-1]) + tau*f(x[j],t[i-1])
    
    matrix_ksi = np.delete(matrix_ksi, 0)
    matrix_ksi = np.delete(matrix_ksi, -1)
                          
    #g[0] = matrix_1[i][0]
    g[0]=(-0.5*0.5*tau*nu(t[i])+h*matrix_ans[i-1][0]/2 + 0.5*tau*0.5*((matrix_ans[i-1][1]-matrix_ans[i-1][0])/h -nu(t[i-1])) + 0.5*tau*h*f(x[0],t[i-1])) / (0.5*h + 0.5*0.5*tau/h)
    
    for j in range(1,g.shape[0]):
        g[j] = (matrix_ksi[j-1]-c*g[j-1])/(b+c*X[j-1]) ### Z
        
    matrix_ans[i][-1] = matrix_1[i][-1]
                          
    for j in reversed(range(1,len(x))): #[1,100]
        matrix_ans[i][j-1] = X[j-1]*matrix_ans[i][j] + g[j-1]
        
#matrix_ans[:,0] = [alpha(t[i]) for i in range(K)] # u(0,t) - заполненние "целых" слоев        
fig = go.Figure()
fig.add_trace(go.Scatter(x=x,y=matrix_k[-1],mode='lines+markers',name='Аналитическое решение'))
fig.add_trace(go.Scatter(x=x,y=matrix_ans[-1],mode='lines+markers',name='Численное решение'))
fig.update_layout(title='Метод Кранка-Николсона')
fig.show()        

In [129]:
t[-1]

3.0

In [131]:
alpha(3.)

20.085536923187668

In [115]:
x[:3]

[0.0, 0.01, 0.02]

In [111]:
f(x[:2][1], 0)

0.505025083542084

In [116]:
f(x[:2][1]+h/2, 0)

0.5075565323078595

In [113]:
h

0.01

In [100]:
delta = matrix_k[-1] - matrix_ans[-1]

In [102]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=x,y=delta,mode='lines+markers',name='невязка'))

In [90]:
len(matrix_ans[1])

101

In [74]:
matrix_ans[:][1]

array([-9.47417919e-02, -7.66361261e-02, -6.17979154e-02, -4.96356615e-02,
       -3.96649250e-02, -3.14889476e-02, -2.47827824e-02, -1.92802958e-02,
       -1.47635226e-02, -1.10539478e-02, -8.00536574e-03, -5.49803248e-03,
       -3.43387614e-03, -1.73257422e-03, -3.28340945e-04,  8.32703934e-04,
        1.79469012e-03,  2.59376174e-03,  3.25952286e-03,  3.81622142e-03,
        4.28371892e-03,  4.67828462e-03,  5.01324609e-03,  5.29952207e-03,
        5.54605892e-03,  5.76018830e-03,  5.94792013e-03,  6.11418275e-03,
        6.26301979e-03,  6.39775165e-03,  6.52110809e-03,  6.63533708e-03,
        6.74229454e-03,  6.84351822e-03,  6.94028901e-03,  7.03368184e-03,
        7.12460844e-03,  7.21385349e-03,  7.30210575e-03,  7.38998533e-03,
        7.47806816e-03,  7.56690878e-03,  7.65706220e-03,  7.74910579e-03,
        7.84366211e-03,  7.94142346e-03,  8.04317932e-03,  8.14984756e-03,
        8.26251086e-03,  8.38245971e-03,  8.51124371e-03,  8.65073320e-03,
        8.80319378e-03,  

In [75]:
pd.DataFrame(matrix_ans)

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.718282
1,-0.094742,-0.076636,-0.061798,-0.049636,-0.039665,-0.031489,-0.024783,-0.019280,-0.014764,-0.011054,...,0.916069,1.115891,1.359847,1.657690,2.021328,2.465303,3.007367,3.669198,4.477264,2.745601
2,-0.081716,-0.081255,-0.077915,-0.072749,-0.066525,-0.059791,-0.052931,-0.046208,-0.039793,-0.033791,...,1.648082,1.787444,1.909026,1.998177,2.034635,1.990766,1.829299,1.500409,0.937994,2.773195
3,-0.127165,-0.109765,-0.096591,-0.086197,-0.077609,-0.070188,-0.063522,-0.057360,-0.051555,-0.046029,...,1.602426,1.684736,1.780205,1.902436,2.073008,2.324914,2.707251,3.291591,4.180559,2.801066
4,-0.114981,-0.114071,-0.109316,-0.102500,-0.094741,-0.086716,-0.078814,-0.071238,-0.064082,-0.057371,...,1.918895,2.055111,2.192466,2.319027,2.413384,2.439489,2.339207,2.021657,1.348132,2.829217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,11.850104,12.161137,12.472770,12.784908,13.097635,13.410984,13.724998,14.039720,14.355193,14.671458,...,47.444165,47.981504,48.523623,49.070570,49.622406,50.179341,50.740865,51.308261,51.879861,52.457326
297,11.969173,12.283493,12.598208,12.913498,13.229366,13.545865,13.863036,14.180922,14.499565,14.819010,...,47.921007,48.463742,49.011302,49.563759,50.121174,50.683461,51.251244,51.823388,52.401650,52.984531
298,12.089677,12.406999,12.724928,13.043374,13.362420,13.682100,14.002459,14.323540,14.645387,14.968043,...,48.402639,48.950828,49.503895,50.061887,50.624867,51.193043,51.765924,52.344748,52.927921,53.517034
299,12.211153,12.531821,12.852897,13.174558,13.496808,13.819702,14.143281,14.467590,14.792672,15.118572,...,48.889113,49.442808,50.001425,50.565038,51.133709,51.707357,52.286587,52.870315,53.460225,54.054889


In [41]:
pd.DataFrame(matrix_k)

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,1.000000,1.010050,1.020201,1.030455,1.040811,1.051271,1.061837,1.072508,1.083287,1.094174,...,2.484323,2.509290,2.534509,2.559981,2.585710,2.611696,2.637944,2.664456,2.691234,2.718282
1,1.010050,1.020201,1.030455,1.040811,1.051271,1.061837,1.072508,1.083287,1.094174,1.105171,...,2.509290,2.534509,2.559981,2.585710,2.611696,2.637944,2.664456,2.691234,2.718282,2.745601
2,1.020201,1.030455,1.040811,1.051271,1.061837,1.072508,1.083287,1.094174,1.105171,1.116278,...,2.534509,2.559981,2.585710,2.611696,2.637944,2.664456,2.691234,2.718282,2.745601,2.773195
3,1.030455,1.040811,1.051271,1.061837,1.072508,1.083287,1.094174,1.105171,1.116278,1.127497,...,2.559981,2.585710,2.611696,2.637944,2.664456,2.691234,2.718282,2.745601,2.773195,2.801066
4,1.040811,1.051271,1.061837,1.072508,1.083287,1.094174,1.105171,1.116278,1.127497,1.138828,...,2.585710,2.611696,2.637944,2.664456,2.691234,2.718282,2.745601,2.773195,2.801066,2.829217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,19.297972,19.491920,19.687817,19.885682,20.085537,20.287400,20.491292,20.697233,20.905243,21.115344,...,47.942386,48.424215,48.910887,49.402449,49.898952,50.400445,50.906978,51.418601,51.935367,52.457326
297,19.491920,19.687817,19.885682,20.085537,20.287400,20.491292,20.697233,20.905243,21.115344,21.327557,...,48.424215,48.910887,49.402449,49.898952,50.400445,50.906978,51.418601,51.935367,52.457326,52.984531
298,19.687817,19.885682,20.085537,20.287400,20.491292,20.697233,20.905243,21.115344,21.327557,21.541903,...,48.910887,49.402449,49.898952,50.400445,50.906978,51.418601,51.935367,52.457326,52.984531,53.517034
299,19.885682,20.085537,20.287400,20.491292,20.697233,20.905243,21.115344,21.327557,21.541903,21.758402,...,49.402449,49.898952,50.400445,50.906978,51.418601,51.935367,52.457326,52.984531,53.517034,54.054889


In [42]:
g

array([1.96917029e-03, 5.60942611e-01, 1.06262594e+00, 1.48801440e+00,
       1.83401949e+00, 2.10829174e+00, 2.32339371e+00, 2.49259552e+00,
       2.62765558e+00, 2.73804628e+00, 2.83097166e+00, 2.91171445e+00,
       2.98405792e+00, 3.05066980e+00, 3.11341227e+00, 3.17357668e+00,
       3.23205498e+00, 3.28946188e+00, 3.34622078e+00, 3.40262353e+00,
       3.45887177e+00, 3.51510535e+00, 3.57142193e+00, 3.62789032e+00,
       3.68455964e+00, 3.74146561e+00, 3.79863477e+00, 3.85608744e+00,
       3.91383972e+00, 3.97190479e+00, 4.03029389e+00, 4.08901694e+00,
       4.14808294e+00, 4.20750032e+00, 4.26727708e+00, 4.32742096e+00,
       4.38793954e+00, 4.44884029e+00, 4.51013059e+00, 4.57181783e+00,
       4.63390935e+00, 4.69641250e+00, 4.75933466e+00, 4.82268320e+00,
       4.88646553e+00, 4.95068909e+00, 5.01536134e+00, 5.08048978e+00,
       5.14608195e+00, 5.21214540e+00, 5.27868775e+00, 5.34571664e+00,
       5.41323975e+00, 5.48126482e+00, 5.54979960e+00, 5.61885192e+00,
      

In [49]:
pd.DataFrame(matrix_1)

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,1.000000,1.01005,1.020201,1.030455,1.040811,1.051271,1.061837,1.072508,1.083287,1.094174,...,2.484323,2.50929,2.534509,2.559981,2.58571,2.611696,2.637944,2.664456,2.691234,2.718282
1,1.010050,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,2.745601
2,1.020201,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,2.773195
3,1.030455,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,2.801066
4,1.040811,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,2.829217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,19.297972,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,52.457326
297,19.491920,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,52.984531
298,19.687817,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,53.517034
299,19.885682,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,54.054889
